# Pipeline: Echocardiogram Analysis

## Manual pipeline for DCM download, decompress and classification of views

In [ ]:
import os
import sys
import pandas as pd
import datetime

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))

import d00_utils.db_utils as db
from d02_intermediate.download_dcm import s3_download_decomp_dcm, dcmdir_to_jpgs_for_classification
from d03_classification.predict_views import run_classify, agg_probabilities, predict_views

In [ ]:
table_name='instances_w_labels'
train_test_ratio=0.5 
downsample_ratio=0.001
train=False
dcm_path = os.path.join(os.path.expanduser('~/data'),'01_raw')
model_path = os.path.join(os.path.expanduser('~/models'), 'view_23_e5_class_11-Mar-2018')

In [ ]:
#dir_name = s3_download_decomp_dcm(dcm_path, table_name, train_test_ratio, downsample_ratio, train)
dir_name = 'test_split50_downsampleby1000'

In [ ]:
dcm_dir = os.path.join(dcm_path, dir_name)
img_dir = os.path.join(os.path.expanduser('~/data'),'02_intermediate', dir_name)

dcmdir_to_jpgs_for_classification(dcm_dir, img_dir)

In [ ]:
run_classify(img_dir, model_path, if_exists='append', feature_dim=1)

In [ ]:
agg_probabilities(if_exists='append')

In [ ]:
predict_views(if_exists='append')

In [ ]:
io_class = db.dbReadWriteClassification()
io_class.list_tables()

In [ ]:
io_class.get_table('predictions')

In [ ]:
evaluate_views(dir_name, model_name, datetime.date.today(), study_filter=None, if_exists="append") 

In [ ]:
io_class.get_table('evaluations')

# Manual pipeline for the segmentation of image files

In [ ]:
from d02_intermediate.create_seg_view import create_seg_view 
from d04_segmentation.segment_view import run_segment
from d04_segmentation.generate_masks import generate_masks
from d04_segmentation.evaluate_masks import evaluate_masks

In [ ]:
dcm_dir = os.path.join(dcm_path, dir_name)
dcm_dir

In [ ]:
model_path = os.path.join(os.path.expanduser('~/models'))
run_segment(dcm_dir, model_path)

In [ ]:
create_seg_view()

In [ ]:
io_views = db.dbReadWriteViews()
io_views.list_tables()

In [ ]:
io_views.get_table('frames_by_volume_mask').head()

In [ ]:
io_views.get_table('chords_by_volume_mask').head()

In [ ]:
generate_masks(dcm_dir)

In [ ]:
io_segmentation = db.dbReadWriteSegmentations()
io_segmentation.list_tables()

In [ ]:
evaluate_masks()